In [47]:
from random import random
import numpy as np
import math
from math import exp

def actF(x):
    act = math.tanh(x)
    return [act,1-act*act]

def cross_entropy_loss(output, target):
    return (output - target)

def preprocess(filename):
    file = open(filename,'r')
    dataset = [line.split() for line in file]
    train_data = [np.array([float(row[0]),float(row[1])]) for row in dataset]
    train_labels = np.zeros((len(train_data),2))
    labels = [int(row[2]) for row in dataset]
    for i in range(len(labels)):
            train_labels[i][labels[i]] = 1    
    return train_data, train_labels

def softmax_norm(output_vec):
    l = [exp(x) for x in output_vec]
    npl = np.array(l)
    mysum = npl.sum()
    Dr = np.array([mysum for i in range(0,len(output_vec))])
    return (npl/Dr)

class NeuralNetwork:
    def __init__(self, n_input, n_hidden, n_output):
        init_weights = [[random() for x in range(n_input+1)] for i in range(n_hidden)]
        init_weights = np.array(init_weights)
        self.hidden_layer = {'W':init_weights}
        init_weights = [[random() for x in range(n_hidden+1)] for i in range(n_output)]
        init_weights = np.array(init_weights)
        self.output_layer = {'W' : init_weights}
        self.layers = 2
        
    def forward_prop(self, input_vector):
        inputs = np.append(input_vector,1)
        w_mat = nn.hidden_layer['W']
        prod = np.matmul(w_mat.reshape(3,4).transpose(),inputs.reshape(3,1))
        act = []
        delta = []
        for i in prod:
            act.append(actF(i)[0])
            delta.append(actF(i)[1])
        nn.hidden_layer['delta'] = np.array(delta)
        new_inputs = act
        new_inputs = np.append(new_inputs,1)
        w_mat = nn.output_layer['W']
        prod = np.matmul(w_mat.reshape(5,2).transpose(),new_inputs.reshape(5,1))
        act = []
        delta = []
        for i in prod:
            act.append(actF(i)[0])
            delta.append(actF(i)[1])
        nn.output_layer['delta'] = np.array(delta)
        return act
    
    def backward_prop(self, loss): 
        loss = loss.reshape(2,1)
        error_grad = [a*b for a,b in zip(loss,nn.output_layer['delta'])]
        learning_rate = 0.5
        error_grad = np.array(error_grad).reshape(2,1)
        nn.output_layer['W'] = nn.output_layer['W'] - learning_rate*error_grad
        
        error_grad1 = nn.hidden_layer['delta'].reshape(4,1) * error_grad.sum()
        nn.hidden_layer['W'] = nn.hidden_layer['W'] - learning_rate* error_grad1
      
    



if __name__ == "__main__":
    train_data, train_labels = preprocess('dataset.txt')
    sum_error=0
    row_count=0
    epochs = 100
    nn = NeuralNetwork(2,4,2)
    for epoch in range(epochs):
        row_count=0
        sum_error =0
        for row in train_data:
            error = []
            output = nn.forward_prop(row)
            output = softmax_norm(output)
            loss = cross_entropy_loss(train_labels[row_count], output)
            if row_count <=1:
#                 print(str(row[0]) + " " + str(row[1]) + " " + str(output[0]) + str(output[1])+ str(loss))
                nn.backward_prop(loss)
                sum_error += sum([(train_labels[row_count] - output)**2 for i in range(len(output))])
                row_count+= 1
#         print("sum:",sum_error)
